# <span style="color:teal">RBFE Network - Analysis</span>


In [ ]:
# import libraries
import BioSimSpace as BSS
from scipy.stats import sem as sem
import sys
import os

if '/home/anna/Documents/cinnabar' not in sys.path:
    sys.path.insert(1, '/home/anna/Documents/cinnabar')
import cinnabar

print(cinnabar.__file__)

if '/home/anna/Documents/code/python' not in sys.path:
    sys.path.insert(1, '/home/anna/Documents/code/python')
import pipeline

print(pipeline.__file__)

from pipeline import *
from pipeline.utils import validate
from pipeline.analysis import *

In [ ]:
protein = "mcl1"

bench_folder = f"/home/anna/Documents/benchmark"
main_dir = f"{bench_folder}/extracted/{protein}"

# choose location for the files
net_file = f"{main_dir}/execution_model/network_lomap.dat"
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
# exp_file = f"{bench_folder}/inputs/experimental/tyk2.yml"
exp_file = f"/home/anna/Documents/benchmark/inputs/experimental/{protein}.yml"

if os.path.exists(f"{main_dir}/outputs_extracted/results"):
    results_folder = f"{main_dir}/outputs_extracted/results"
elif os.path.exists(f"{main_dir}/outputs/results"):
    results_folder = f"{main_dir}/outputs/results"
else:
    raise ValueError(f"results directory not found in the {main_dir}. please make sure results were written using the analysis script previously in the pipeline")

output_folder = validate.folder_path(f"{main_dir}/analysis", create=True)


In [ ]:
all_analysis_object = analysis_network(results_folder,
                                       exp_file=exp_file,
                                       net_file=net_file,
                                       output_folder=output_folder,
                                       analysis_ext=ana_file
                                        )

# can add any other results files
# all_analysis_object.compute_other_results(file_name=None, name=None)
all_analysis_object.compute(cycle_closure=False)

There are different options for plotting:
bar (pert or lig)
scatter (pert or lig) - can be plotted using cinnabar
eng vs eng
other results (eg from other workflows)
outliers
histograms (for the error of the runs, sem, legs)

In [ ]:
# bar
all_analysis_object.plot_bar_lig()
all_analysis_object.plot_bar_pert()

#scatter
all_analysis_object.plot_scatter_lig()
all_analysis_object.plot_scatter_pert()
all_analysis_object.plot_scatter_lig(use_cinnabar=True)
all_analysis_object.plot_scatter_pert(use_cinnabar=True)
for eng in all_analysis_object.engines:
    all_analysis_object.plot_scatter_lig(engine=eng)
    all_analysis_object.plot_scatter_pert(engine=eng)

    # outliers
    all_analysis_object.plot_outliers(engine=eng)
    all_analysis_object.plot_outliers(engine=eng, pert_val="val")

all_analysis_object.plot_histogram_legs(free_bound="free")
all_analysis_object.plot_histogram_legs(free_bound="bound")
all_analysis_object.plot_histogram_runs()
all_analysis_object.plot_histogram_sem()
all_analysis_object.plot_histogram_sem(pert_val="pert")

plot other results

In [ ]:
other_results = sorted(glob.glob(f"{bench_folder}/inputs/other_computed/raw/*"))
for file in other_results:
    new_file = f"{file.replace(file.split('/')[-1], '')[:-1].replace('/raw','')}/{other_results.index(file)}_mcl1.csv"
    
    with open(new_file, "w") as write_file:
        writer = csv.writer(write_file, delimiter=",")

        writer.writerow(["lig_0", "lig_1", "freenrg",
                        "error", "engine", "analysis", "method"])
    
        with open(file, "r") as read_file:
            for line in read_file:
                if "Calculated" in line:
                    pass
                else:
                    lig_0 = line.split(',')[0].replace('protein_','')
                    lig_1 = line.split(',')[1].replace('protein_','')
                    freenrg = line.split(',')[2]
                    error = line.split(',')[3]

                writer.writerow([lig_0, lig_1, freenrg, error, "MBAR","other"])


In [ ]:
all_analysis_object.compute_other_results(sorted(glob.glob(f"{bench_folder}/inputs/other_computed/other/*.csv")), name="other")
all_analysis_object.plot_other_results(name="other", engine="AMBER", pert_val="pert")

In [ ]:
shared = []

for key in all_analysis_object.cinnabar_calc_pert_dict["AMBER"].keys():
    if key in all_analysis_object.cinnabar_calc_pert_dict["other"].keys():
        shared.append(key)

print(len(shared))
shared

plot just one engine to check

In [ ]:
all_analysis_object.plot_scatter_lig(engine="AMBER", use_cinnabar=True)
all_analysis_object.plot_scatter_pert(engine="AMBER", use_cinnabar=True)

plot engine vs engine

In [ ]:
all_analysis_object.plot_eng_vs_eng("SOMD","AMBER")

plot convergence

In [ ]:
ex_outputs_folder = f"{main_dir}/outputs_extracted"

converg_obj = plot_convergence(ex_outputs_folder,
                               perturbations=all_analysis_object.perturbations,
                               engines=all_analysis_object.engines,
                               file_ext=ana_file
                               )

converg_obj.plot_convergence_all()

remove a perturbation

In [ ]:
eng = "AMBER"
perturbations = ["lig_30~lig_40", "lig_27~lig_48"]
# ['lig_ejm42~lig_ejm44', 'lig_ejm42~lig_ejm49', 'lig_ejm42~lig_ejm45']
# ['lig_ejm42~lig_ejm49', 'lig_ejm45~lig_ejm55', 'lig_ejm43~lig_ejm50'] #gromacs
# ['lig_ejm45~lig_ejm53', 'lig_ejm44~lig_ejm45', 'lig_ejm31~lig_ejm49'] #somd

for pert in perturbations:
    try:
        all_analysis_object.remove_perturbations(pert)
    except:
        pass
    
print(all_analysis_object.perturbations)
all_analysis_object.compute()


# plot scatter plots
# all_analysis_object.plot_scatter_lig(engine=eng, use_cinnabar=True)
# all_analysis_object.plot_scatter_pert(engine="AMBER")
# all_analysis_object.plot_outliers(engine=eng, outliers=3, pert_val="pert")

statistical analysis vs exp, other results, each engine to the other etc

In [ ]:
all_analysis_object.calc_mae(pert_val="pert", engines=["GROMACS","AMBER"])
# all_analysis_object.calc_mae(pert_val="val")

In [ ]:
# old

# import networkanalysis
# import experiments
# import stats

# exp_dicts = res_obj._get_exp_fwf(fwf_path='/home/anna/Documents/september_2022_workshops/freenrgworkflows/networkanalysis/')
# for key in exp_dicts[0]:
#     print(f"{key} : {exp_dicts[0][key][0]}, {exp_dicts[0][key][1]}")

free_fwf_dict = res_obj._get_ana_fwf(fwf_path='/home/anna/Documents/september_2022_workshops/freenrgworkflows/networkanalysis/', engine="SOMD")
for key in free_fwf_dict:
    print(f"{key} : {free_fwf_dict[key][0]}, {free_fwf_dict[key][1]}")

# fwf_stats = res_obj._get_stats_fwf(fwf_path='/home/anna/Documents/september_2022_workshops/freenrgworkflows/networkanalysis/', engine="SOMD")
